# A Recommender System for Groceries Contractor

# A Recommender System for Groceries Contractor

In [1]:
# importing libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
import geopy.geocoders # convert an address into latitude and longitude values

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

### Postal Codes in Toronto

In [7]:
# Loading the dataset which is about postal codes in Toronto
# This dataset was created in week 3. 
df_toronto = pd.read_csv('toronto_part2.csv')
df_toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81153,-79.19552
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17520
3,M1G,Scarborough,Woburn,43.76820,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76969,-79.23944


### Create a Map of Toronto City (with its Postal Codes' Regions）

In [9]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

### Focusing on the "Scarorough" Borough in Toronto (its neighborhoods)

In [11]:
# df_toronto['Borough'] == 'Scarborough'

# selecting only neighborhoods regarding to "Scarborough" borough.
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough']
scarborough_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81153,-79.19552
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17520
3,M1G,Scarborough,Woburn,43.76820,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76969,-79.23944


### Create a Map of Scarborough and Its Neighbourhoods

In [13]:
address_scar = 'Scarborough, Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of "Scarborough" are: {}, {}.'.format(latitude_scar, longitude_scar))

map_Scarborough = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=11.5)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_Scarborough)  
    
map_Scarborough

The geograpical coordinate of "Scarborough" are: 43.773077, -79.257774.


In [14]:
def foursquare_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;

In [15]:
# @hiddel_cell
CLIENT_ID = '0MJA3NYYG3U2ZY1LTZN2OYEHS3Y3WVSON2GBSO3IL4EDYVIR' # your Foursquare ID
CLIENT_SECRET = 'WGWSAF2TKVUQPE3PD0N3EOITFVBY5EYP1VCZI3BMUG0ROUS5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Crawling Internet (in fact only Foursquare database) for Venues in the Neighborhoods inside "Scarborough"

In [17]:
print('Crawling different neighborhoods inside "Scarborough"')
Scarborough_foursquare_dataset = foursquare_crawler(list(scarborough_data['Postalcode']),
                                                   list(scarborough_data['Neighborhood']),
                                                   list(scarborough_data['Latitude']),
                                                   list(scarborough_data['Longitude']),)

Crawling different neighborhoods inside "Scarborough"
1.
Data is Obtained, for the Postal Code M1B (and Neighborhoods Malvern, Rouge) SUCCESSFULLY.
2.
Data is Obtained, for the Postal Code M1C (and Neighborhoods Rouge Hill, Port Union, Highland Creek) SUCCESSFULLY.
3.
Data is Obtained, for the Postal Code M1E (and Neighborhoods Guildwood, Morningside, West Hill) SUCCESSFULLY.
4.
Data is Obtained, for the Postal Code M1G (and Neighborhoods Woburn) SUCCESSFULLY.
5.
Data is Obtained, for the Postal Code M1H (and Neighborhoods Cedarbrae) SUCCESSFULLY.
6.
Data is Obtained, for the Postal Code M1J (and Neighborhoods Scarborough Village) SUCCESSFULLY.
7.
Data is Obtained, for the Postal Code M1K (and Neighborhoods Kennedy Park, Ionview, East Birchmount Park) SUCCESSFULLY.
8.
Data is Obtained, for the Postal Code M1L (and Neighborhoods Golden Mile, Clairlea, Oakridge) SUCCESSFULLY.
9.
Data is Obtained, for the Postal Code M1M (and Neighborhoods Cliffside, Cliffcrest, Scarborough Village West) 

### saving result

In [18]:
import pickle
with open("Scarborough_foursquare_dataset.txt", "wb") as fp:   #Pickling
    pickle.dump(Scarborough_foursquare_dataset, fp)
print('Received Data from Internet is Saved to Computer.')

Received Data from Internet is Saved to Computer.


In [19]:
with open("Scarborough_foursquare_dataset.txt", "rb") as fp:   # Unpickling
    Scarborough_foursquare_dataset = pickle.load(fp)
# print(type(Scarborough_foursquare_dataset))
# Scarborough_foursquare_dataset

### Cleaning the RAW Data Received from Foursquare Database

In [20]:
# This function is created to connect to the saved list which is the received database. It will extract each venue 
# for every neighborhood inside the database

def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
    # print(result_df)
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venuse in Coordination "{}" Posal Code and "{}" Negihborhood(s) is:'.format(postal_code, neigh))
        print(len(neigh_dict['Crawling_result']))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']
            
            
            # print({'Postal Code': postal_code, 'Neighborhood': neigh, 
            #                   'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
            #                   'Venue': name, 'Venue Summary': summary, 
            #                   'Venue Category': cat, 'Distance': dist})
            
            result_df = result_df.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
            # print(result_df)
    
    return(result_df)

In [21]:
scarborough_venues = get_venue_dataset(Scarborough_foursquare_dataset)

Number of Venuse in Coordination "M1B" Posal Code and "Malvern, Rouge" Negihborhood(s) is:
7
Number of Venuse in Coordination "M1C" Posal Code and "Rouge Hill, Port Union, Highland Creek" Negihborhood(s) is:
4
Number of Venuse in Coordination "M1E" Posal Code and "Guildwood, Morningside, West Hill" Negihborhood(s) is:
18
Number of Venuse in Coordination "M1G" Posal Code and "Woburn" Negihborhood(s) is:
17
Number of Venuse in Coordination "M1H" Posal Code and "Cedarbrae" Negihborhood(s) is:
23
Number of Venuse in Coordination "M1J" Posal Code and "Scarborough Village" Negihborhood(s) is:
12
Number of Venuse in Coordination "M1K" Posal Code and "Kennedy Park, Ionview, East Birchmount Park" Negihborhood(s) is:
28
Number of Venuse in Coordination "M1L" Posal Code and "Golden Mile, Clairlea, Oakridge" Negihborhood(s) is:
20
Number of Venuse in Coordination "M1M" Posal Code and "Cliffside, Cliffcrest, Scarborough Village West" Negihborhood(s) is:
13
Number of Venuse in Coordination "M1N" Pos

### Showing Venues for Each Neighborhood in Scarborough

In [22]:
scarborough_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M1B,"Malvern, Rouge",43.81153,-79.19552,Canadiana exhibit,This spot is popular,Zoo Exhibit,736
1,M1B,"Malvern, Rouge",43.81153,-79.19552,Wendy’s,This spot is popular,Fast Food Restaurant,535
2,M1B,"Malvern, Rouge",43.81153,-79.19552,Ontrack Services,This spot is popular,Financial or Legal Service,615
3,M1B,"Malvern, Rouge",43.81153,-79.19552,Grizzly Bear Exhibit,This spot is popular,Zoo Exhibit,634
4,M1B,"Malvern, Rouge",43.81153,-79.19552,Upper Rouge Trail,This spot is popular,Trail,772


In [23]:
scarborough_venues.to_csv('scarborough_venues.csv')

In [24]:
scarborough_venues = pd.read_csv('scarborough_venues.csv')

In [25]:
# Some Summary Information about Neighborhoods inside "Scarborough"
neigh_list = list(scarborough_venues['Neighborhood'].unique())
print('Number of Neighborhoods inside Scarborough:')
print(len(neigh_list))
print('List of Neighborhoods inside Scarborough:')
neigh_list

Number of Neighborhoods inside Scarborough:
16
List of Neighborhoods inside Scarborough:


['Malvern, Rouge',
 'Rouge Hill, Port Union, Highland Creek',
 'Guildwood, Morningside, West Hill',
 'Woburn',
 'Cedarbrae',
 'Scarborough Village',
 'Kennedy Park, Ionview, East Birchmount Park',
 'Golden Mile, Clairlea, Oakridge',
 'Cliffside, Cliffcrest, Scarborough Village West',
 'Birch Cliff, Cliffside West',
 'Dorset Park, Wexford Heights, Scarborough Town Centre',
 'Wexford, Maryvale',
 'Agincourt',
 "Clarks Corners, Tam O'Shanter, Sullivan",
 "Milliken, Agincourt North, Steeles East, L'Amoreaux East",
 "Steeles West, L'Amoreaux West"]

In [26]:
neigh_venue_summary = scarborough_venues.groupby('Neighborhood').count()
neigh_venue_summary.drop(columns = ['Unnamed: 0']).head()

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
Neighborhood,,,,,,,
Agincourt,50,50,50,50,50,50,50
"Birch Cliff, Cliffside West",12,12,12,12,12,12,12
Cedarbrae,23,23,23,23,23,23,23
"Clarks Corners, Tam O'Shanter, Sullivan",43,43,43,43,43,43,43
"Cliffside, Cliffcrest, Scarborough Village West",13,13,13,13,13,13,13


In [27]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

print('Here is the list of different categories:')
list(scarborough_venues['Venue Category'].unique())

There are 108 uniques categories.
Here is the list of different categories:


['Zoo Exhibit',
 'Fast Food Restaurant',
 'Financial or Legal Service',
 'Trail',
 'Other Great Outdoors',
 'Burger Joint',
 'Breakfast Spot',
 'Italian Restaurant',
 'Park',
 'Liquor Store',
 'Food & Drink Shop',
 'Pizza Place',
 'Smoothie Shop',
 'Coffee Shop',
 'Pharmacy',
 'Sports Bar',
 'Discount Store',
 'Gym / Fitness Center',
 'Supermarket',
 'Athletics & Sports',
 'Gymnastics Gym',
 'Convenience Store',
 'Restaurant',
 'Filipino Restaurant',
 'Indian Restaurant',
 'Juice Bar',
 'Chinese Restaurant',
 'Vietnamese Restaurant',
 'Thrift / Vintage Store',
 'Department Store',
 'Sandwich Place',
 'Bakery',
 'Caribbean Restaurant',
 'Hakka Restaurant',
 'Music Store',
 'Thai Restaurant',
 'Bank',
 'Gas Station',
 'Fried Chicken Joint',
 'Flower Shop',
 'German Restaurant',
 'Bus Line',
 'Ice Cream Shop',
 'Big Box Store',
 'Train Station',
 'Hobby Shop',
 'Grocery Store',
 'Light Rail Station',
 'Rental Car Location',
 'Vegetarian / Vegan Restaurant',
 'Beer Store',
 'Pub',
 'Soccer

### One-hot Encoding the "categroies" Column into Every Unique Categorical Feature.

In [28]:
scarborough_onehot = pd.get_dummies(data = scarborough_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
scarborough_onehot.head()

,Unnamed: 0,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,African Restaurant,American Restaurant,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bank,Beer Store,Big Box Store,Bistro,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Butcher,Café,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,German Restaurant,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Hardware Store,Hobby Shop,Hong Kong Restaurant,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motorcycle Shop,Music Store,Noodle House,Other Great Outdoors,Park,Pet Store,Pharmacy,Pizza Place,Pool,Pool Hall,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Sports Bar,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Zoo Exhibit
0,0,M1B,"Malvern, Rouge",43.81153,-79.19552,Canadiana exhibit,This spot is popular,736,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,M1B,"Malvern, Rouge",43.81153,-79.19552,Wendy’s,This spot is popular,535,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,M1B,"Malvern, Rouge",43.81153,-79.19552,Ontrack Services,This spot is popular,615,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,M1B,"Malvern, Rouge",43.81153,-79.19552,Grizzly Bear Exhibit,This spot is popular,634,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,4,M1B,"Malvern, Rouge",43.81153,-79.19552,Upper Rouge Trail,This spot is popular,772,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


### Manually Selecting (Subsetting) Related Features for the Groceries Contractor

In [32]:
# This list is created manually 
important_list_of_features = [
 
 'Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',

 'African Restaurant',
 'American Restaurant',
 'Asian Restaurant',

 
 'BBQ Joint',
 
 'Bakery',
 
 
 
 
 
 'Breakfast Spot',

 'Burger Joint',
 
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 
 'Diner',


 'Fast Food Restaurant',
 'Filipino Restaurant',
 'Fish Market',
 'Food & Drink Shop',
 'Fried Chicken Joint',
 
 'Greek Restaurant',
 'Grocery Store',
 
 'Hakka Restaurant',
 
 'Hong Kong Restaurant',
 
 'Indian Restaurant',

 'Italian Restaurant',
 'Japanese Restaurant',
 'Korean Restaurant',
 'Latin American Restaurant',



 'Malay Restaurant',
 
 'Mediterranean Restaurant',
 
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 
 'Noodle House',
 
 'Pizza Place',
 
 'Restaurant',
 'Sandwich Place',
 'Seafood Restaurant',
 
 'Sushi Restaurant',
 'Taiwanese Restaurant',
 
 'Thai Restaurant',
 
 'Vegetarian / Vegan Restaurant',
 
 'Vietnamese Restaurant',
 'Wings Joint']

In [33]:
scarborough_onehot = scarborough_onehot[important_list_of_features].drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby(
    'Neighborhood').sum()


scarborough_onehot.head()

,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Diner,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Fried Chicken Joint,Greek Restaurant,Grocery Store,Hakka Restaurant,Hong Kong Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Noodle House,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agincourt,0,0,2,0,1,1,0,2,0,9,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,1,1,0,0,1,1,3,2,1,1,0,0,0,0,0
"Birch Cliff, Cliffside West",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Cedarbrae,0,0,0,0,2,0,1,0,1,1,0,0,0,0,0,1,0,0,1,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
"Clarks Corners, Tam O'Shanter, Sullivan",0,0,0,0,2,0,0,1,1,2,0,3,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,2,0,1,0,0,1,1,0,3,0
"Cliffside, Cliffcrest, Scarborough Village West",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0


Integrating Different Restaurants and Different Joints
(Assuming Different Resaturants Use the Same Raw Groceries)
This Assumption is made for simplicity and due to not having very large dataset about neighborhoods.

In [34]:
feat_name_list = list(scarborough_onehot.columns)
restaurant_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Restaurant') != (-1):
        restaurant_list.append(value)
        
scarborough_onehot['Total Restaurants'] = scarborough_onehot[restaurant_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = restaurant_list)


feat_name_list = list(scarborough_onehot.columns)
joint_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Joint') != (-1):
        joint_list.append(value)
        
scarborough_onehot['Total Joints'] = scarborough_onehot[joint_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = joint_list)

### Showing the Fully-Processed DataFrame about Neighborhoods inside Scarborrough.This Dataset is Ready for any Machine Learning Algorithm.

In [35]:
scarborough_onehot

,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Grocery Store,Noodle House,Pizza Place,Sandwich Place,Total Restaurants,Total Joints
Neighborhood,,,,,,,,,,,
Agincourt,1,1,0,0,0,1,1,1,2,23,0
"Birch Cliff, Cliffside West",0,0,1,0,0,0,0,0,0,0,0
Cedarbrae,2,0,0,0,0,0,0,0,0,8,2
"Clarks Corners, Tam O'Shanter, Sullivan",2,0,0,0,0,0,0,2,1,15,1
"Cliffside, Cliffcrest, Scarborough Village West",0,0,0,0,0,0,0,1,1,1,0
"Dorset Park, Wexford Heights, Scarborough Town Centre",1,0,0,0,0,0,0,3,0,10,2
"Golden Mile, Clairlea, Oakridge",2,0,1,0,0,0,0,0,0,2,0
"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,1,0,3,0
"Kennedy Park, Ionview, East Birchmount Park",0,0,0,0,0,2,0,2,1,5,1


### Run k-means to Cluster Neighborhoods into 5 Clusters

In [36]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(scarborough_onehot)

### show center of each cluster 

In [37]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = scarborough_onehot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Grocery Store,Noodle House,Pizza Place,Sandwich Place,Total Restaurants,Total Joints,Total Sum
G4,1.000000,1.000000,0.000000,0.0,0.000000,1.0,1.0,1.000000,2.000000,23.000000,0.000000,30.000000
G2,1.500000,0.500000,0.000000,0.5,0.000000,1.0,0.0,3.000000,0.500000,14.000000,1.500000,22.500000
G5,1.666667,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.333333,0.000000,8.000000,2.000000,13.000000
G1,0.666667,0.333333,0.000000,0.0,0.000000,1.0,0.0,2.000000,1.000000,5.333333,0.333333,10.666667
G3,0.285714,0.142857,0.285714,0.0,0.142857,0.0,0.0,0.428571,0.428571,1.571429,0.142857,3.428571


### Result:
#### Best Group is G5;
#### Second Best Group is G1;
#### Third Best Group is G4;
#### Inserting "kmeans.labels_" into the Original Scarborough DataFrame
#### Finding the Corresponding Group for Each Neighborhood.

In [43]:
neigh_summary = pd.DataFrame([scarborough_onehot.index, 1 + kmeans.labels_]).T
neigh_summary.columns = ['Neighborhood', 'Group']
neigh_summary

,Neighborhood,Group
0,Agincourt,4
1,"Birch Cliff, Cliffside West",3
2,Cedarbrae,5
3,"Clarks Corners, Tam O'Shanter, Sullivan",2
4,"Cliffside, Cliffcrest, Scarborough Village West",3
5,"Dorset Park, Wexford Heights, Scarborough Town...",5
6,"Golden Mile, Clairlea, Oakridge",3
7,"Guildwood, Morningside, West Hill",3
8,"Kennedy Park, Ionview, East Birchmount Park",1
9,"Malvern, Rouge",3


### result: Best Neighborhood Are...

In [41]:
neigh_summary[neigh_summary['Group'] == 5]

,Neighborhood,Group
2,Cedarbrae,5
5,"Dorset Park, Wexford Heights, Scarborough Town...",5
10,"Milliken, Agincourt North, Steeles East, L'Amo...",5


In [44]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 5]['Neighborhood'])[0]
scarborough_venues[scarborough_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M1H',
 'Neighborhood': 'Cedarbrae',
 'Neighborhood Latitude': 43.76969000000002,
 'Neighborhood Longitude': -79.23943999999996}